# Downloading data from AWS bucket - XETRA, with sync

In [26]:
pip install aws

     |████████████████████████████████| 55 kB 4.2 MB/s eta 0:00:011
     |████████████████████████████████| 212 kB 6.7 MB/s eta 0:00:01
     |████████████████████████████████| 171 kB 20.8 MB/s eta 0:00:01
     |████████████████████████████████| 349 kB 7.6 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 14.6 MB/s eta 0:00:01
  Created wheel for aws: filename=aws-0.2.5-py3-none-any.whl size=7459 sha256=b889d5ccf08c7dfa40c8b7594e3f610a3ad4808cfdcc5482d5103a2507ccdf9d
  Stored in directory: /Users/generous/Library/Caches/pip/wheels/c0/90/24/3a59385cf082b75bf2c2548ad09dd36c68111b0ea5b16a6b0b
Successfully built aws
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import os, sys

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [5]:
spark = SparkSession\
    .builder\
    .appName("Xetra")\
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/10 14:16:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
from_date = '2017-06-17'
till_date = '2022-03-18'

In [14]:
local_data_folder = 'deutsche-boerse-xetra-pds-test' # do not end in /
input_bucket = 's3://deutsche-boerse-xetra-pds' # do not end in /

In [8]:
download_script = 'download_data.sh'

In [11]:
dates = list(pd.date_range(from_date, till_date, freq='D').strftime('%Y-%m-%d'))
dates

['2017-06-17',
 '2017-06-18',
 '2017-06-19',
 '2017-06-20',
 '2017-06-21',
 '2017-06-22',
 '2017-06-23',
 '2017-06-24',
 '2017-06-25',
 '2017-06-26',
 '2017-06-27',
 '2017-06-28',
 '2017-06-29',
 '2017-06-30',
 '2017-07-01',
 '2017-07-02',
 '2017-07-03',
 '2017-07-04',
 '2017-07-05',
 '2017-07-06',
 '2017-07-07',
 '2017-07-08',
 '2017-07-09',
 '2017-07-10',
 '2017-07-11',
 '2017-07-12',
 '2017-07-13',
 '2017-07-14',
 '2017-07-15',
 '2017-07-16',
 '2017-07-17',
 '2017-07-18',
 '2017-07-19',
 '2017-07-20',
 '2017-07-21',
 '2017-07-22',
 '2017-07-23',
 '2017-07-24',
 '2017-07-25',
 '2017-07-26',
 '2017-07-27',
 '2017-07-28',
 '2017-07-29',
 '2017-07-30',
 '2017-07-31',
 '2017-08-01',
 '2017-08-02',
 '2017-08-03',
 '2017-08-04',
 '2017-08-05',
 '2017-08-06',
 '2017-08-07',
 '2017-08-08',
 '2017-08-09',
 '2017-08-10',
 '2017-08-11',
 '2017-08-12',
 '2017-08-13',
 '2017-08-14',
 '2017-08-15',
 '2017-08-16',
 '2017-08-17',
 '2017-08-18',
 '2017-08-19',
 '2017-08-20',
 '2017-08-21',
 '2017-08-

In [12]:
! mkdir -p -v {local_data_folder}

In [13]:
! ls -la


total 8376
drwxr-xr-x    16 generous  staff      512 May 10 14:16 .
drwx------+    8 generous  staff      256 May 10 11:14 ..
-rw-r--r--@    1 generous  staff    10244 May  4 17:01 .DS_Store
drwxr-xr-x    15 generous  staff      480 May  4 16:56 .git
-rw-r--r--     1 generous  staff      141 May  4 16:56 .gitignore
drwxr-xr-x     7 generous  staff      224 May  3 08:54 .ipynb_checkpoints
-rw-r--r--@    1 generous  staff   457569 Apr 26 20:49 ASTP_week1_aula1.ipynb
-rw-r--r--@    1 generous  staff    92035 May 10 14:13 Download-AWS-Xetra copy.ipynb
-rw-r--r--@    1 generous  staff    92035 May 10 14:13 Download-AWS-Xetra.ipynb
-rw-r--r--     1 generous  staff   728571 Apr 26 18:21 code.ipynb
drwxr-xr-x@ 1739 generous  staff    55648 Apr  7 15:26 deutsche-boerse-xetra-pds
drwxr-xr-x   904 generous  staff    28928 Apr 27 15:29 deutsche-boerse-xetra-pds-2017-06-17-till-2022-03-18.parquet
drwxr-xr-x     2 generous  staff       64 May 10 14:16 deutsche-boerse-xetra-pds-test
-rw-r--r--     1 

Create the script

In [15]:
with open(download_script, 'w') as f:
    f.write("#!/bin/bash\n")
    f.write("\nset -euo pipefail\n")
    f.write("\n# This script was generated to download data for multiple days\n")
    for date in dates:
        success_file =  os.path.join(local_data_folder, date, 'success')

        f.write("""
if [ ! -f {success_file} ]; then

    echo "Getting PDS dataset for date {date}"        
    mkdir -p {local_data_folder}/{date}
    aws s3 sync s3://deutsche-boerse-xetra-pds/{date} {local_data_folder}/{date} --no-sign-request
    touch {success_file}            
else
    echo "PDS dataset for date {date} already exists"
fi\n""".format(success_file=success_file, date=date, local_data_folder=local_data_folder))


In [16]:
! chmod +x {download_script} 

In [17]:
! ls -la

total 9880
drwxr-xr-x    17 generous  staff      544 May 10 14:18 .
drwx------+    8 generous  staff      256 May 10 11:14 ..
-rw-r--r--@    1 generous  staff    10244 May  4 17:01 .DS_Store
drwxr-xr-x    15 generous  staff      480 May  4 16:56 .git
-rw-r--r--     1 generous  staff      141 May  4 16:56 .gitignore
drwxr-xr-x     8 generous  staff      256 May 10 14:17 .ipynb_checkpoints
-rw-r--r--@    1 generous  staff   457569 Apr 26 20:49 ASTP_week1_aula1.ipynb
-rw-r--r--@    1 generous  staff    94875 May 10 14:17 Download-AWS-Xetra copy.ipynb
-rw-r--r--@    1 generous  staff    92035 May 10 14:13 Download-AWS-Xetra.ipynb
-rw-r--r--     1 generous  staff   728571 Apr 26 18:21 code.ipynb
drwxr-xr-x@ 1739 generous  staff    55648 Apr  7 15:26 deutsche-boerse-xetra-pds
drwxr-xr-x   904 generous  staff    28928 Apr 27 15:29 deutsche-boerse-xetra-pds-2017-06-17-till-2022-03-18.parquet
drwxr-xr-x     2 generous  staff       64 May 10 14:16 deutsche-boerse-xetra-pds-test
-rwxr-xr-x     1 

In [23]:
! cat download_data.sh
# ! head -n 20 {download_script} 

#!/bin/bash

set -euo pipefail

# This script was generated to download data for multiple days

if [ ! -f deutsche-boerse-xetra-pds-test/2017-06-17/success ]; then

    echo "Getting PDS dataset for date 2017-06-17"        
    mkdir -p deutsche-boerse-xetra-pds-test/2017-06-17
    aws s3 sync s3://deutsche-boerse-xetra-pds/2017-06-17 deutsche-boerse-xetra-pds-test/2017-06-17 --no-sign-request
    touch deutsche-boerse-xetra-pds-test/2017-06-17/success            
else
    echo "PDS dataset for date 2017-06-17 already exists"
fi

if [ ! -f deutsche-boerse-xetra-pds-test/2017-06-18/success ]; then

    echo "Getting PDS dataset for date 2017-06-18"        
    mkdir -p deutsche-boerse-xetra-pds-test/2017-06-18
    aws s3 sync s3://deutsche-boerse-xetra-pds/2017-06-18 deutsche-boerse-xetra-pds-test/2017-06-18 --no-sign-request
    touch deutsche-boerse-xetra-pds-test/2017-06-18/success            
else
    echo "PDS dataset for date 2017-06-18 already exists"
fi

if [ ! -f deutsche-boers

    aws s3 sync s3://deutsche-boerse-xetra-pds/2019-01-21 deutsche-boerse-xetra-pds-test/2019-01-21 --no-sign-request
    touch deutsche-boerse-xetra-pds-test/2019-01-21/success            
else
    echo "PDS dataset for date 2019-01-21 already exists"
fi

if [ ! -f deutsche-boerse-xetra-pds-test/2019-01-22/success ]; then

    echo "Getting PDS dataset for date 2019-01-22"        
    mkdir -p deutsche-boerse-xetra-pds-test/2019-01-22
    aws s3 sync s3://deutsche-boerse-xetra-pds/2019-01-22 deutsche-boerse-xetra-pds-test/2019-01-22 --no-sign-request
    touch deutsche-boerse-xetra-pds-test/2019-01-22/success            
else
    echo "PDS dataset for date 2019-01-22 already exists"
fi

if [ ! -f deutsche-boerse-xetra-pds-test/2019-01-23/success ]; then

    echo "Getting PDS dataset for date 2019-01-23"        
    mkdir -p deutsche-boerse-xetra-pds-test/2019-01-23
    aws s3 sync s3://deutsche-boerse-xetra-pds/2019-01-23 deutsche-boerse-xetra-pds-test/2019-01-23 --no-sign-request
  

    echo "PDS dataset for date 2021-12-08 already exists"
fi

if [ ! -f deutsche-boerse-xetra-pds-test/2021-12-09/success ]; then

    echo "Getting PDS dataset for date 2021-12-09"        
    mkdir -p deutsche-boerse-xetra-pds-test/2021-12-09
    aws s3 sync s3://deutsche-boerse-xetra-pds/2021-12-09 deutsche-boerse-xetra-pds-test/2021-12-09 --no-sign-request
    touch deutsche-boerse-xetra-pds-test/2021-12-09/success            
else
    echo "PDS dataset for date 2021-12-09 already exists"
fi

if [ ! -f deutsche-boerse-xetra-pds-test/2021-12-10/success ]; then

    echo "Getting PDS dataset for date 2021-12-10"        
    mkdir -p deutsche-boerse-xetra-pds-test/2021-12-10
    aws s3 sync s3://deutsche-boerse-xetra-pds/2021-12-10 deutsche-boerse-xetra-pds-test/2021-12-10 --no-sign-request
    touch deutsche-boerse-xetra-pds-test/2021-12-10/success            
else
    echo "PDS dataset for date 2021-12-10 already exists"
fi

if [ ! -f deutsche-boerse-xetra-pds-test/2021-12-11/succes

In the terminal, execute the script that has been created (this way is more effective I guess):

% 
    ./download_data.sh

In [30]:
!./download_data.sh

Getting PDS dataset for date 2017-06-17
Traceback (most recent call last):
  File "/Users/generous/opt/anaconda3/bin/aws", line 5, in <module>
    from aws.main import main
  File "/Users/generous/opt/anaconda3/lib/python3.9/site-packages/aws/main.py", line 23
    print '%(name)s: %(endpoint)s' % {
          ^
SyntaxError: invalid syntax


In [19]:
local_data_folder

'deutsche-boerse-xetra-pds-test'

In [20]:
# read all data

df_raw = spark.read.load(local_data_folder, 
                         format="csv", 
                         header=True, 
                         pathGlobFilter="*.csv",
                         recursiveFileLookup=True  
                        )

AnalysisException: Unable to infer schema for CSV. It must be specified manually.

In [10]:
df_raw.printSchema()

root
 |-- ISIN: string (nullable = true)
 |-- Mnemonic: string (nullable = true)
 |-- SecurityDesc: string (nullable = true)
 |-- SecurityType: string (nullable = true)
 |-- Currency: string (nullable = true)
 |-- SecurityID: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- StartPrice: string (nullable = true)
 |-- MaxPrice: string (nullable = true)
 |-- MinPrice: string (nullable = true)
 |-- EndPrice: string (nullable = true)
 |-- TradedVolume: string (nullable = true)
 |-- NumberOfTrades: string (nullable = true)



In [11]:
df_raw.count()

98240766

In [12]:
df_raw.show(5, vertical=True, truncate=False)

-RECORD 0---------------------------------
 ISIN           | DE000A0DJ6J9            
 Mnemonic       | S92                     
 SecurityDesc   | SMA SOLAR TECHNOL.AG    
 SecurityType   | Common stock            
 Currency       | EUR                     
 SecurityID     | 2504287                 
 Date           | 2022-03-07              
 Time           | 08:00                   
 StartPrice     | 29.78                   
 MaxPrice       | 30.02                   
 MinPrice       | 29.5                    
 EndPrice       | 29.5                    
 TradedVolume   | 7338                    
 NumberOfTrades | 26                      
-RECORD 1---------------------------------
 ISIN           | DE000A0D6554            
 Mnemonic       | NDX1                    
 SecurityDesc   | NORDEX SE O.N.          
 SecurityType   | Common stock            
 Currency       | EUR                     
 SecurityID     | 2504290                 
 Date           | 2022-03-07              
 Time      

In [13]:
df_dates = df_raw.groupBy('Date').count()

In [14]:
df_dates.show()

+----------+------+
|      Date| count|
+----------+------+
|2022-03-07|166777|
|2022-02-22|132730|
|2022-01-21|130734|
|2020-02-28|147557|
|2022-03-09|149453|
|2022-01-24|168133|
|2020-11-09|136201|
|2022-02-24|174259|
|2022-01-25|140619|
|2021-11-29|128138|
|2021-11-26|152833|
|2022-03-08|148191|
|2022-02-14|140500|
|2022-03-04|145965|
|2021-01-28|126651|
|2022-03-15|125134|
|2022-03-14|125322|
|2022-02-25|136414|
|2020-02-24|116928|
|2020-03-25|127204|
+----------+------+
only showing top 20 rows



In [15]:
df_dates.count()

1218

In [16]:
df_dates_year = df_raw.groupBy(year('Date')).count()

In [17]:
df_dates_year.count()

6

In [18]:
df_dates_year.show()

+----------+--------+
|year(Date)|   count|
+----------+--------+
|      2020|22908728|
|      2021|25586838|
|      2022| 6794791|
|      2018|16946437|
|      2019|18004474|
|      2017| 7999498|
+----------+--------+



In [21]:
filename = local_data_folder + '-' + from_date + '-till-' + till_date + '.parquet'
filename

'deutsche-boerse-xetra-pds-2017-06-17-till-2022-03-18.parquet'

In [29]:
print(type(df_raw))

<class 'pyspark.sql.dataframe.DataFrame'>


In [30]:
df_raw.write.mode("overwrite").parquet(filename)


22/04/07 16:11:31 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/04/07 16:11:36 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/04/07 16:11:36 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/04/07 16:11:36 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/04/07 16:11:39 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/04/07 16:11:39 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/04/07 16:11:42 WARN MemoryManager: Total allocation exceeds 95.00% 

22/04/07 16:12:09 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/04/07 16:12:10 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/04/07 16:12:10 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/04/07 16:12:10 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/04/07 16:12:11 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/04/07 16:12:11 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/04/07 16:12:12 WARN MemoryManager: Total allocation exceeds 95.00% 

22/04/07 16:12:28 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/04/07 16:12:28 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/04/07 16:12:29 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/04/07 16:12:30 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/04/07 16:12:30 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/04/07 16:12:30 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/04/07 16:12:30 WARN MemoryManager: Total allocation exceeds 95.00% 

22/04/07 16:12:46 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/04/07 16:12:46 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/04/07 16:12:47 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/04/07 16:12:48 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/04/07 16:12:48 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/04/07 16:12:48 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/04/07 16:12:48 WARN MemoryManager: Total allocation exceeds 95.00% 

22/04/07 16:13:07 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/04/07 16:13:07 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/04/07 16:13:07 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/04/07 16:13:08 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/04/07 16:13:08 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/04/07 16:13:09 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/04/07 16:13:09 WARN MemoryManager: Total allocation exceeds 95.00% 

22/04/07 16:13:22 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/04/07 16:13:22 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/04/07 16:13:22 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/04/07 16:13:22 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/04/07 16:13:23 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
